In [1]:
# !pip -q install utils
# !pip -q install openai
# !pip -q install gradio
# !pip -q install langchain
# !pip -q install streamlit
# !pip -q install python-dotenv
# !pip -q install streamlit_float
# !pip -q install audio_recorder_streamlit


!npm install localtunnel


added 7 packages, and audited 416 packages in 3s

40 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities


npm notice 
npm notice New major version of npm available! 9.2.0 -> 10.6.0
npm notice Changelog: <https://github.com/npm/cli/releases/tag/v10.6.0>
npm notice Run `npm install -g npm@10.6.0` to update!
npm notice 


In [5]:
import os
import uuid
import time
import openai
# import pygame
# import librosa
import base64
# import gradio as gr
import streamlit as st

from openai import OpenAI
from base64 import b64decode
# from google.colab import userdata
from typing import Optional, Tuple
from IPython.display import Javascript, Audio
# from google.colab import userdata, output

from langchain import LLMChain # LLM
from langchain.chat_models import ChatOpenAI # ChatModel
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
# from langchain_core.messages import HumanMessage, SystemMessage
from langchain.memory import ConversationBufferWindowMemory

# Transcribe Class (ASR)

In [21]:
# RECORD = """
#     const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
#     const b2text = blob => new Promise(resolve => {
#       const reader = new FileReader()
#       reader.onloadend = e => resolve(e.srcElement.result)
#       reader.readAsDataURL(blob)
#     })
#     var record = time => new Promise(async resolve => {
#       stream = await navigator.mediaDevices.getUserMedia({ audio: true })
#       recorder = new MediaRecorder(stream)
#       chunks = []
#       recorder.ondataavailable = e => chunks.push(e.data)
#       recorder.start()
#       await sleep(time)
#       recorder.onstop = async ()=>{
#         blob = new Blob(chunks)
#         text = await b2text(blob)
#         resolve(text)
#       }
#       recorder.stop()
#     })
# """

In [4]:
%%writefile transcribe.py

import os
import uuid

from openai import OpenAI

class Transcribe:

    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)
        self.path = './chatbot/transcribe'

        if not os.path.exists(self.path):
            os.makedirs(self.path)

    def speech_to_text(self, audio):
        with open(audio, 'rb') as audio_file:

            transcription = self.client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file,
                response_format="text"
            )

        return transcription

    def save_file(self, audio_bytes):
        unique_filename = str(uuid.uuid4()) + ".mp3"
        filepath = os.path.join(self.path, unique_filename)

        with open(filepath,'wb') as f:
            f.write(audio_bytes)

        return filepath

    def delete_file(self):
        for file in os.listdir(self.path):
            if file.endswith(".mp3"):
                os.remove(os.path.join(self.path, file))


Overwriting transcribe.py


# LangChain

In [5]:
%%writefile llmchain.py

# from langchain_core.memory import BaseMemory
# from langchain_core.prompts.prompt import PromptTemplate
from langchain import LLMChain
from langchain_core.prompts import PromptTemplate
# from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatOpenAI


from langchain import LLMChain  # LLM
from langchain import LLMChain
from langchain.memory import ConversationBufferWindowMemory
import streamlit as st

@st.cache_resource
class Langchain:
    def __init__(self, api_key, model_name='gpt-4'):
        self.llm = ChatOpenAI(temperature=0, api_key=api_key, model_name=model_name)
        self.template = """

                        This is OpenAI

                        Previous conversation:
                        {history}

                        Human: {question}

                        Assistant:

                        """
        self.prompt = PromptTemplate(
            input_variables = ["history", "question"],
            template = self.template
        )
        self.memory = ConversationBufferWindowMemory(k=5)
        self.chain = self.create_chain()

    def create_chain(self):
        return LLMChain(
            llm=self.llm,
            prompt=self.prompt,
            # verbose=True,
            memory=self.memory
        )

    def chat(self, question):
        response = self.chain.run({"question": question})

        return response

Overwriting llmchain.py


# Voice Class (TTS)

In [6]:
%%writefile voice.py

import base64
import os
import uuid

import streamlit as st
from openai import OpenAI

class Voice:

    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)
        self.path = './chatbot/voice'

        if not os.path.exists(self.path):
            os.makedirs(self.path)

    def text_to_speech(self, transcript):
        response = self.client.audio.speech.create(
            model="tts-1-hd",
            voice="echo",
            input=transcript,
        )

        unique_filename = str(uuid.uuid4()) + ".mp3"
        filepath = os.path.join(self.path, unique_filename)
        response.stream_to_file(filepath)

        return filepath

    def delete_file(self):
        for file in os.listdir(self.path):
            if file.endswith(".mp3"):
                os.remove(os.path.join(self.path, file))

    def autoplay_audio(self, file_path: str):
        with open(file_path, "rb") as f:
            data = f.read()
        b64 = base64.b64encode(data).decode("utf-8")
        md = f"""
        <audio autoplay>
        <source src="data:audio/mp3;base64,{b64}" type="audio/mp3">
        </audio>
        """
        st.markdown(md, unsafe_allow_html=True)

Overwriting voice.py


# Main

In [7]:
%%writefile .env

openai_api_key=""

Overwriting .env


- https://typethepipe.com/post/streamlit-chat-conversational-app-st-chat_message/
- https://bijukunjummen.medium.com/chat-application-using-streamlit-and-text-bison-05024f939827

In [8]:
%%writefile app.py

import os
import streamlit as st

from streamlit_float import *
from dotenv import load_dotenv
# from google.colab import userdata
from audio_recorder_streamlit import audio_recorder

import llmchain
import transcribe
import voice

load_dotenv()
key = os.getenv("openai_api_key")

transcribe = transcribe.Transcribe(api_key=key)
langchain = llmchain.Langchain(api_key=key)
voice = voice.Voice(api_key=key)

float_init()

def initialize_session_state():
    if "messages" not in st.session_state:
        st.session_state.messages = [
            {"role": "assistant", "content": "Yoii gess"}
        ]

initialize_session_state()

st.title("COBAGESS")

# Create footer container for the microphone
footer_container = st.container()
with footer_container:
    audio_bytes = audio_recorder()

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])


if audio_bytes:
    # Write the audio bytes to a file
    with st.spinner("Transcribing..."):

        file_path = transcribe.save_file(audio_bytes)
        transcript = transcribe.speech_to_text(file_path)

        if transcript:
            st.session_state.messages.append({"role": "user", "content": transcript})
            with st.chat_message("user"):
                st.write(transcript)

if st.session_state.messages[-1]["role"] != "assistant":

    with st.chat_message("assistant"):
        # contoh_response = 'Hello'
        # st.write(contoh_response)
        with st.spinner("Thinking🤔..."):
            data = st.session_state.messages
            last_user_index = None
            for i, item in enumerate(data):
                if item["role"] == "user":
                    last_user_index = i

            # Mengambil nilai content dari peran "user" terakhir jika ditemukan
            if last_user_index is not None:
                last_user_content = data[last_user_index]["content"]

            response = langchain.chat(last_user_content)

            audio_file = voice.text_to_speech(response)
            voice.autoplay_audio(audio_file)

        st.write(response)
            # st.write(file_path)
        st.session_state.messages.append({"role": "assistant", "content": response})
        voice.delete_file()
        transcribe.delete_file()


# Float the footer container and provide CSS to target it with
footer_container.float("bottom: 0rem;")

Overwriting app.py


In [31]:
!streamlit run /content/app.py &>/content/logs.txt & curl https://loca.lt/mytunnelpassword

34.127.41.171

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.026s
your url is: https://fancy-towns-relate.loca.lt


In [66]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

--2024-04-30 01:43:37--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  19.4MB/s    in 0.7s    

2024-04-30 01:43:38 (19.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]



In [68]:
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [69]:
get_ipython().system_raw('./ngrok http 8501 &')

In [70]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Execute the next cell and the go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.10/json/__init__.py", line 293, in load
    return loads(fp.read(),
  File "/usr/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [71]:
!streamlit run /content/app.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.80.190:8501

  Stopping...
  Stopping...
